This notebook demonstrates fine-tuning seven pretrained models ['Xception','InceptionV3','ResNet101','VGG19','DenseNet201',MobileNetV2',EfficientNetB2'] for classification of retinal diseases on OCT images. 

In [ ]:
# First import the necessary libraries

import os
import random

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
#import functons_cnn_OCT.ipynb
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ipynb.fs.full.functions_cnn_OCT import *

In [ ]:
# Call the root directory of OCT images
base_dir = os.path.join("/kaggle/input/kermany2018/oct2017/OCT2017 /")
print('Base directory --> ', os.listdir(base_dir))

In [ ]:
# Call the train, validation and test directory paths and store them appropriately

train_dir = os.path.join(base_dir + "train/")
print("Train Directory --> ", os.listdir(train_dir))

validation_dir = os.path.join(base_dir + "val/")
print("Validation Directory --> ", os.listdir(validation_dir))

test_dir = os.path.join(base_dir + "test/")
print("Test Directory --> ", os.listdir(test_dir))

In [ ]:
#call the display_image function from functions.ipynb to display the random images
display_image(train_dir)

In [ ]:
#generate the model using generate_model function from functions.ipynb
#input the pretrained models from ['Xception','InceptionV3','ResNet101','VGG19','DenseNet201',MobileNetV2',EfficientNetB2']
model=generate_model('DenseNet201')

In [ ]:
#prepare the metrics and data generators to train the model
#list the metrics 
metrics_list = ['accuracy',
                tf.keras.metrics.AUC(),
                tfa.metrics.CohenKappa(num_classes = 4),
                tfa.metrics.F1Score(num_classes = 4),
                tf.keras.metrics.Precision(), 
                tf.keras.metrics.Recall()]

#generate train_generator
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 100)

#generate validation_generator
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 16)

#generate test_generator
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 44)

In [ ]:
#train the model
#input model architecture, train dataset, validationd dataset, number_of_epcohs, optimizer and metrics list
#here the models were trained using 100 epochs with adam optimizer, six performance measures
trained_model,trained_history=train_model(model,train_generator,validation_generator,100, 'adam', metrics_list)

In [ ]:
#Once the training process is finished, plot the learning curves
plot_acc_loss(trained_model,trained_history)

In [ ]:
#evaluate the model using evaluate_model_function from functions.ipynb
#this function will output accuracy, AUC, Cohen Kappa, F score, precision and recall
evaluate_model(trained_model,trained_history,test_generator)

In [ ]:
#test the model using test_model function
#this will output confusion matrix, classificaiotn report and performance metrics
test_model(trained_model,test_generator)

In [ ]:
#Import COde to save Pretrained Models
#after run this code, it will show the file available. can download manually
from IPython.display import FileLink, FileLinks
FileLinks('.') #lists all downloadable files on server

In [ ]:
#save the train model for future 
trained_model.save("/kaggle/working/DenseNet201_best_model.h5")

In [ ]:
#here is another way to save it
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Best_DenseNet.h5")
print("Saved model to disk")